# Gather Data

In [9]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#for api calls
from bs4 import BeautifulSoup
import requests
import json
import time

#personal api and aws keys
import config

import query_helper
# import websrcape_wiki_table

## Create climbs db

In [ ]:
query_helper.create_db('climbs')

## Create route tables

In [ ]:
route_table = """
CREATE TABLE routes (
    id int NOT NULL,
    name varchar(100),
    type varchar(30),
    rating varchar(20),
    stars float,
    starVotes int,
    pitches int,
    location varchar(255),
    url varchar(255),
    longitude float,
    latitude float,
    PRIMARY KEY (id)
    );
"""

In [ ]:
query_helper.create_table(route_table)

In [ ]:
route_describe_table = """
CREATE TABLE route_description (
    id int NOT NULL,
    info TEXT,
    PRIMARY KEY (id)
    );
"""

In [ ]:
query_helper.create_table(route_describe_table)

## Use Mountain Project Api

### Get GPS Coordinates

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('waypoints (1).xml')
root = tree.getroot()

In [ ]:
coordinates = []
for child in root:
    coordinates.append(child.attrib)

In [ ]:
len(coordinates)

### Using GPS Coordinates to get routes and store to db

In [ ]:
def mp_gps_call(url_params, api_key):
# https://www.mountainproject.com/data/get-routes-for-lat-lon?lat=40.03&lon=-105.25&maxDistance=10&minDiff=5.6&maxDiff=5.10&key=112327568-ec557ff70496df7d0a43436ab9df1201
    url = 'https://www.mountainproject.com/data/get-routes-for-lat-lon'
#     headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, params=url_params)
    
    return response.json()

#### Loop over list of GPS coordinates, get routes and insert into db

In [ ]:
for i,coord in enumerate(coordinates):
    ##assign new lat long and use api
    api_key = config.api_key
    lat = coord['lat']   #32.904600
    lon = coord['lon']  #-116.818400
    maxResults =500
    maxDistance = 10
    minDiff = '5.0'
    maxDiff = '5.16'

    url_params = {  'lat': str(lat),
                    'lon': str(lon),
                    'maxResults': maxResults,
    #                 'maxDistance' : str(maxDistance),
    #                 'minDiff': str(minDiff),
    #                 'maxDiff': str(maxDiff)
                    'key': api_key
                 }
    # call function and return 5000 route details
    data = mp_gps_call(url_params, api_key)
    
    #check that call successful 
    if data['success']:
        routes = data['routes']
        
        #store route data in mysql
        query_helper.route_to_tup(routes)
        
        #track our progress
        f = open("so_far_we_got.txt", "a")
        f.write(f'{i} iteration {lat},{lon}')
        f.close()
        
    #print error message if something goes wrong
    else:
        f'Error!! stopped at {i} iteration {lat},{lon}'
        f = open("stopped_at.txt", "a")
        f.write(f'Error!! stopped at {i} iteration {lat},{lon}')
        f.close()
        break
        
    #sleep for a bit to avoid being blocked
    time.sleep(3)


## Webscrape Mountain Project for  Route Descriptions and Reviews

### Get route ids from MySql db

In [2]:
id_df = query_helper.query_to_df("SELECT id FROM climbs.routes;")
id_df.head()

,id
0,105714722
1,105714728
2,105714731
3,105714734
4,105714737


In [ ]:
# import config
# import json
# import requests
# api_key = config.api_key
# import time 
# import datetime
# import mysql.connector
# from mysql.connector import errorcode
# import pandas as pd


# #creates connection, all functions will start by calling this
# def connect():
#     global cnx
#     cnx = mysql.connector.connect(
#     host = config.host,
#     user = config.user,
#     passwd = config.password,
#     database = 'climbs')
#     global cursor
#     cursor = cnx.cursor()

In [10]:
def get_route_info(id_):
    
    #get page content
    web_address = f'https://www.mountainproject.com/route/{id_}'
    
    page = requests.get(web_address)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #grab section of interest
    info =soup.find_all(class_='fr-view')
    
    describe=[]
    for i in range(len(info)):
        describe.append(info[i].get_text())
    
    return describe

In [ ]:
# def route_info_to_db(info):
#     connect()
#     insert_statement = """INSERT IGNORE INTO route_description(
#                     id,
#                     info)
#                     VALUES (%s,%s)"""

#     cursor.execute(insert_statement, info)
#     cnx.commit()
#     cursor.close()
#     cnx.close()
    

In [ ]:
for climb_id in remaining:
    ##assign new lat long and use api

    # call function and return 5000 route details
    description = get_route_info(climb_id)
    
    query_helper.route_info_to_db((climb_id, '-'.join(description)))

    #sleep for a bit to avoid being blocked
#     time.sleep(1)

In [54]:
df_describe = query_helper.query_to_df("SELECT id FROM climbs.route_description")

In [55]:
remaining=[]
x=[]
for i in id_df.id:
    if i in df_describe.id.values:
        x.append(i)
    else:
        remaining.append(i)

In [56]:
len(remaining)

16851

### Route info and description

In [ ]:
import json
import requests
from bs4 import BeautifulSoup



def get_mush_table(web_address):
web_address = 'https://www.mountainproject.com/route/105792316/robbins-crack'
    
page = requests.get(web_address)
soup = BeautifulSoup(page.content, 'html.parser')


soup
soup_save = soup
# ### IF IMPORT IS MESSED UP LOOK HERE. ASSUMES mush Box is always second
# info = soup.find_all('table',{"class":"infobox"})

# #get mushroom name 
# name = [i.string for i in info[1].findAll('i')]

# #get mushroom info
# divs = info[1].findAll("td")
# stuff= divs[1:]
# text = []
# for i in stuff:
#     text.append(i.get_text())

# name_and_info = name+text

In [ ]:
info =soup_save.find_all(class_='fr-view')
info

In [ ]:
describe=[]
for i in range(len(info)):
    describe.append(info[i].get_text())

In [ ]:
describe

In [ ]:
# def get_mush_table(web_address):
web_address = 'https://www.mountainproject.com/route/117346798/fangbanger'
    
page = requests.get(web_address)
soup = BeautifulSoup(page.content, 'html.parser')


soup
soup_save = soup

In [ ]:
info =soup_save.find_all(class_='fr-view')
info

In [ ]:
describe=[]
for i in range(len(info)):
    describe.append(info[i].get_text())
describe

### Comments

In [ ]:
# def get_mush_table(web_address):
web_address = 'https://www.mountainproject.com/route/105883232/the-swiftness'
    
page = requests.get(web_address)
soup = BeautifulSoup(page.content, 'html.parser')


soup
soup_save = soup

In [ ]:
page.content

In [ ]:
soup_save


In [ ]:
info =soup_save.find_all(class_='comment-body')
info

In [ ]:
# info = soup.find_all('table',{"class":"infobox"})